# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Perform cross validation on a model to determine optimal model performance
- Compare training and testing errors to determine if model is over or underfitting

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to pre-process below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features['B']
logdis = np.log(boston_features['DIS'])
loglstat = np.log(boston_features['LSTAT'])

# Min-Max scaling
boston_features['B'] = (b-min(b))/(max(b)-min(b))
boston_features['DIS'] = (logdis-min(logdis))/(max(logdis)-min(logdis))

# Standardization
boston_features['LSTAT'] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [2]:
X = boston_features[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']]
y = pd.DataFrame(boston.target, columns = ['target'])
type(X)

pandas.core.frame.DataFrame

### Train-test split

Perform a train-test split with a test set of 20%.

In [4]:
# Import train_test_split from sklearn.model_selection
from sklearn.linear_model import LinearRegression  # linear regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [5]:
# Split the data into training and test sets (assign 20% to test set)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2)

In [6]:
# A brief preview of train-test split
print(len(X_train), len(X_test), len(y_train), len(y_test))

404 102 404 102


### Fit the model

Fit a linear regression model and apply the model to make predictions on test set

In [7]:
linreg=LinearRegression()
model = linreg.fit(X_train,y_train)

### Residuals and MSE

Calculate the residuals and the mean squared error on the test set

In [8]:
y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)
# Calculate mean square errors
train_mse = mean_squared_error(y_train,y_hat_train)
test_mse = mean_squared_error(y_test,y_hat_test) 

In [9]:
train_mse,test_mse

(21.214986954691646, 24.498483144059698)

## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function `kfolds()` that splits a dataset into k evenly sized pieces. If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [19]:
def kfolds(data, k):
    # Force data as pandas DataFrame
    # add 1 to fold size to account for leftovers 
    data = pd.DataFrame(data)
    num_observations= len(data)
    fold_size = num_observations//k
    leftovers = num_observations%k
    folds = []
    start_obs=0
    for fold_n in range (1,k+1):
        if fold_n <= leftovers:
            # fold size =fold_size+1 to account for leftovers
            fold = data.iloc[start_obs : start_obs+fold_size+1]
            folds.append(fold)
            start_obs += fold_size+1
        else:
             # fold size =fold_size (no more leftovers)
            fold = data.iloc[start_obs : start_obs+fold_size]
            folds.append(fold)
            start_obs += fold_size
            
    return folds

### Apply it to the Boston Housing data

In [20]:
# Make sure to concatenate the data again: predictors + dependent variable
bos_data = pd.concat([X,y],axis=1)
bos_data[7:10],X[7:10],y[7:10],

(   CHAS     RM       DIS         B     LSTAT  target
 7   0.0  6.172  0.700059  1.000000  0.968416    27.1
 8   0.0  5.631  0.709276  0.974104  1.712312    16.5
 9   0.0  6.004  0.743201  0.974305  0.779802    18.9,
    CHAS     RM       DIS         B     LSTAT
 7   0.0  6.172  0.700059  1.000000  0.968416
 8   0.0  5.631  0.709276  0.974104  1.712312
 9   0.0  6.004  0.743201  0.974305  0.779802,
    target
 7    27.1
 8    16.5
 9    18.9)

In [29]:
# Apply kfolds() to bos_data with 5 folds
bos_folds= kfolds(bos_data, 5)

In [30]:
print(len(bos_data),len(bos_data)//5,len(bos_data)%5)
bos_folds

506 101 1


[     CHAS     RM       DIS         B     LSTAT  target
 0     0.0  6.575  0.542096  1.000000 -1.275260    24.0
 1     0.0  6.421  0.623954  1.000000 -0.263711    21.6
 2     0.0  7.185  0.623954  0.989737 -1.627858    34.7
 3     0.0  6.998  0.707895  0.994276 -2.153192    33.4
 4     0.0  7.147  0.707895  1.000000 -1.162114    36.2
 ..    ...    ...       ...       ...       ...     ...
 97    0.0  8.069  0.475885  1.000000 -1.555067    38.7
 98    0.0  7.820  0.475885  0.991502 -1.829759    43.8
 99    0.0  7.416  0.475885  1.000000 -0.912931    33.2
 100   0.0  6.727  0.379096  0.994604 -0.213444    27.5
 101   0.0  6.781  0.390808  0.996672 -0.555806    26.5
 
 [102 rows x 6 columns],      CHAS     RM       DIS         B     LSTAT  target
 102   0.0  6.405  0.369415  0.177720 -0.012136    18.6
 103   0.0  6.137  0.369415  0.993873  0.378596    19.3
 104   0.0  6.167  0.321174  0.989384  0.235000    20.1
 105   0.0  5.851  0.262627  0.992814  0.717270    19.5
 106   0.0  5.836  0.2

### Perform a linear regression for each fold and calculate the training and test error

Perform linear regression on each and calculate the training and test error: 

In [31]:
type(bos_folds),type(bos_folds[0]),

(list, pandas.core.frame.DataFrame)

In [33]:
list(enumerate(bos_folds))

[(0,      CHAS     RM       DIS         B     LSTAT  target
  0     0.0  6.575  0.542096  1.000000 -1.275260    24.0
  1     0.0  6.421  0.623954  1.000000 -0.263711    21.6
  2     0.0  7.185  0.623954  0.989737 -1.627858    34.7
  3     0.0  6.998  0.707895  0.994276 -2.153192    33.4
  4     0.0  7.147  0.707895  1.000000 -1.162114    36.2
  ..    ...    ...       ...       ...       ...     ...
  97    0.0  8.069  0.475885  1.000000 -1.555067    38.7
  98    0.0  7.820  0.475885  0.991502 -1.829759    43.8
  99    0.0  7.416  0.475885  1.000000 -0.912931    33.2
  100   0.0  6.727  0.379096  0.994604 -0.213444    27.5
  101   0.0  6.781  0.390808  0.996672 -0.555806    26.5
  
  [102 rows x 6 columns]),
 (1,      CHAS     RM       DIS         B     LSTAT  target
  102   0.0  6.405  0.369415  0.177720 -0.012136    18.6
  103   0.0  6.137  0.369415  0.993873  0.378596    19.3
  104   0.0  6.167  0.321174  0.989384  0.235000    20.1
  105   0.0  5.851  0.262627  0.992814  0.717270    

In [36]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    # train = all folds but the fold number n
    train = pd.concat([fold for i,fold in enumerate(bos_folds) if i!= n])
    test = bos_folds[n]
    # Fit a linear regression model
    model = linreg.fit(train[X.columns],train[y.columns])
    # Evaluate Train and Test errors
    # Calculate predictions on training and test sets
    y_hat_train = model.predict(train[X.columns])
    y_hat_test = model.predict(test[X.columns])
    # Calculate mean square errors
    train_errs.append (mean_squared_error(train[y.columns],y_hat_train))
    test_errs.append (mean_squared_error(test[y.columns],y_hat_test)) 
    
print(train_errs)
print(test_errs)

[24.195577370388623, 23.032087348477972, 19.74507285797898, 15.31710138425119, 22.32997280754659]
[13.405144922008185, 17.44401680068205, 37.03271139002071, 58.27954384784228, 26.097988757148393]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [37]:
from sklearn.model_selection import cross_val_score
cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring='neg_mean_squared_error')


Next, calculate the mean of the MSE over the 5 cross-validation and compare and contrast with the result from the train-test split case.

In [38]:
cv_5_results

array([-13.40514492, -17.4440168 , -37.03271139, -58.27954385,
       -26.09798876])

##  Summary 

Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are robust and always use it whenever possible! 